## Import Dependencies in Python

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
get_ipython().run_line_magic('matplotlib', 'inline')
from pmdarima.arima import auto_arima
import warnings
warnings.filterwarnings('ignore')
import datetime
import quandl
from plotnine import *
from statsmodels.tsa.vector_ar.var_model import VAR
import math 
import plotly.graph_objects as go

## Step 1) Import daily data

In [2]:
df = pd.read_csv('daily_data.csv')
df = df.rename(columns={"Unnamed: 0": "date"})
df.set_index('date')
df.index = pd.to_datetime(df['date'],yearfirst=True)
df['date'] = pd.to_datetime(df['date'],yearfirst=True)
df

,date,WTI,BRENT,exxon_close,chevron_close,conoco_close,eog_close,valero_close,baker_close,WTI_increase,...,chevron_increase,chevron_decrease,conoco_increase,conoco_decrease,eog_increase,eog_decrease,valero_increase,valero_decrease,baker_increase,baker_decrease
date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,1990-01-02,22.88,21.20,12.500000,17.281250,9.767180,3.046875,3.456353,25.875000,0,...,0,0,0,0,0,0,0,0,0,0
1990-01-03,1990-01-03,23.81,22.65,12.375000,17.000000,9.576602,3.031250,3.513483,25.500000,1,...,0,1,0,1,0,1,1,0,0,1
1990-01-04,1990-01-04,23.41,22.50,12.250000,16.781250,9.386023,2.953125,3.570612,24.875000,0,...,0,1,0,1,0,1,1,0,0,1
1990-01-05,1990-01-05,23.07,23.13,12.187500,16.531250,9.290733,2.906250,3.599177,24.750000,0,...,0,1,0,1,0,1,1,0,0,1
1990-01-08,1990-01-08,21.64,21.38,12.375000,16.687500,9.481312,2.859375,3.542048,25.000000,0,...,1,0,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-22,2020-05-22,33.49,33.80,44.599998,90.279999,43.279999,52.279999,65.680000,15.120000,0,...,0,1,0,1,0,1,0,1,0,1
2020-05-26,2020-05-26,34.70,33.95,45.910000,93.300003,43.669998,51.439999,68.699997,16.010000,1,...,1,0,1,0,0,1,1,0,1,0
2020-05-27,2020-05-27,32.80,32.73,46.240002,93.900002,44.669998,52.730000,70.180000,16.559999,0,...,1,0,1,0,1,0,1,0,1,0


## Step 2) Create separate dfs for WTI, BRENT, and the exogenous lagged stock price variables

In [3]:
WTI_ts=df[['WTI']]
BRENT_ts=df[['BRENT']]
exog_ts=df[['WTI_increase','BRENT_increase',
 'exxon_increase',
 'chevron_increase',
 'conoco_increase',
 'eog_increase',
 'valero_increase',
 'baker_increase']]

## Step 3) Spliting the datasets into training and test datasets 

In [4]:
split_date = pd.datetime(2019,1,1)
WTI_train = WTI_ts.loc[WTI_ts.index < split_date]
WTI_test = WTI_ts.loc[WTI_ts.index >= split_date]
BRENT_train = BRENT_ts.loc[BRENT_ts.index < split_date]
BRENT_test = BRENT_ts.loc[BRENT_ts.index >= split_date]
exog_train = exog_ts.loc[exog_ts.index < split_date]
exog_test = exog_ts.loc[exog_ts.index >= split_date]

## Step 4) Running ARIMA model with previous data stock and oil prices as predictors of the WTI and BRENT data

In [ ]:
WTI_arima_model =  auto_arima(WTI_train,exogenous=exog_train, start_p=0, d=1, start_q=0, 
                          max_p=5, max_d=5, max_q=5, start_P=0, 
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=12, seasonal=True, 
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=26146.409, Time=11.05 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=24044.482, Time=26.61 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=115.91 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=26106.227, Time=8.76 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=23246.474, Time=76.68 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=22758.982, Time=134.36 sec
 ARIMA(1,1,0)(4,1,0)[12]             : AIC=22487.725, Time=249.45 sec
 ARIMA(1,1,0)(5,1,0)[12]             : AIC=22365.366, Time=365.62 sec


#### Results: The best fitting ARIMA model for WTI with our exogenous features is a (2, 1, 3) model. The summary of the WTI ARIMA model details below.

In [ ]:
WTI_arima_model.summary()

In [ ]:
BRENT_arima_model =  auto_arima(BRENT_train,exogenous=exog_train, start_p=0, d=1, start_q=0, 
                          max_p=5, max_d=5, max_q=5, start_P=0, 
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=12, seasonal=True, 
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True)

#### Results: The best fitting ARIMA model for BRENT with our exogenous features is a (1, 1, 2) model. The summary of the BRENT ARIMA model details below.

In [ ]:
#Summary of the model
BRENT_arima_model.summary()

## Step 5) Compare the predicted oil prices from the ARIMA models against the observed values from our test datasets to test the accuracy of our models

In [ ]:
#Obtain the ARIMA model's predicted values and 95% CI from 2019 to 2020

WTI_prediction, WTI_confint = WTI_arima_model.predict(n_periods = 351, exogenous = exog_test, return_conf_int=True)
WTI_prediction = pd.DataFrame(WTI_prediction,index=WTI_test.index)
WTI_prediction.columns = ['WTI_predicted_oil_price']
WTI_lower_series = pd.Series(WTI_confint[:, 0], index=WTI_test.index)
WTI_upper_series = pd.Series(WTI_confint[:, 1], index=WTI_test.index)

#Calculate the model fit and accuracy scores of ARIMA model for predicitng WTI test data from 2019 to 2020

WTI_accuracy = metrics.r2_score(WTI_test, WTI_prediction)
WTI_ARIMA_MSE = metrics.mean_squared_error(WTI_test, WTI_prediction)

In [ ]:
#Visualising the predictions and original data using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=WTI_train.index,
        y=WTI_train["WTI"], mode='lines',
        name='WTI Train'))
fig.add_trace(go.Scatter(x=WTI_test.index,
        y=WTI_test["WTI"],mode='lines',
        name='"WTI Test'))
fig.add_trace(go.Scatter(x=WTI_predictions.index,
        y=WTI_prediction,mode='lines',
        name='WTI ARIMA Predicted'))
fig.update_layout(
    title="f'ARIMA  (2, 1, 3) model for WTI Daily Oil Price Over Time: MSE = {WTI_ARIMA_MSE}",
    xaxis_title="Year",
    yaxis_title="Price($)/Barrel")
fig.show()

In [ ]:
plt.xlabel('Year')
plt.ylabel('Price($)/Barrel')
plt.xlim([datetime.date(2015, 1, 1), datetime.date(2020, 6, 1)])
plt.plot(WTI_train["WTI"],label="WTI Daily Training")
plt.plot(WTI_test["WTI"],label="WTI Daily Test")
plt.plot(WTI_prediction,label="WTI Predicted")

plt.title(f'ARIMA (2, 1, 3) model for WTI Daily Oil Price Over Time: MSE {WTI_ARIMA_MSE}')
plt.legend(loc = 'Left corner')
plt.show()

In [ ]:
print(f'The Model fit for the ARIMA model using the daily WTI oil price data is: R2 = {WTI_accuracy} and MSE = {WTI_ARIMA_MSE}')


In [ ]:
#Obtain the ARIMA model's predicted values and 95% CI from 2019 to 2020

BRENT_prediction, BRENT_confint = BRENT_arima_model.predict(n_periods = 351, return_conf_int=True)
BRENT_prediction = pd.DataFrame(BRENT_prediction,index=BRENT_test.index)
BRENT_prediction.columns = ['BRENT_predicted_oil_price']
BRENT_lower_series = pd.Series(BRENT_confint[:, 0], index=WTI_test.index)
BRENT_upper_series = pd.Series(BRENT_confint[:, 1], index=WTI_test.index)

#Calculate the model fit and accuracy scores of ARIMA model for predicitng WTI test data from 2019 to 2020

BRENT_accuracy = metrics.r2_score(BRENT_test, BRENT_prediction)
BRENT_ARIMA_MSE = metrics.mean_squared_error(BRENT_test, BRENT_prediction)

In [ ]:
plt.xlabel('Year')
plt.ylabel('Price($)/Barrel')
plt.plot(BRENT_train,label="BRENT Daily Training")
plt.plot(BRENT_test,label="BRENT Daily Test")
plt.xlim([datetime.date(2015, 1, 1), datetime.date(2020, 6, 1)])
plt.plot(BRENT_prediction,label="BRENT Daily Predicted")
plt.fill_between(BRENT_lower_series.index, 
                     BRENT_lower_series, 
                     BRENT_upper_series, 
                     alpha=0.25, label='95%CI')
plt.title(f'ARIMA (1, 1, 0) model for BRENT Daily Oil Price Over Time: MSE = {BRENT_ARIMA_MSE}')
plt.legend(loc = 'Left corner')
plt.show()

In [ ]:
print(f'The Model fit for the BRENT ARIMA model using the daily oil price data is: R2 = {BRENT_accuracy} and MSE = {BRENT_ARIMA_MSE}')
